In [11]:
import polars as pl
from ortools.sat.python import cp_model

armor = pl.read_parquet("data/armor_pieces.parquet")
charms = pl.read_parquet("data/charms.parquet")
jewels = pl.read_parquet("data/jewels.parquet")
talents = pl.read_parquet("data/talents.parquet")
weapons = pl.read_parquet("data/weapons.parquet")


In [6]:
model = cp_model.CpModel()

In [54]:
use_equipement_vars = {}
talents_not_aggregated = {}
talents_aggregated = {}

use_equipement_vars["charm"] = {}

# Create non aggregated talent list
for charm in charms.iter_rows():
    charm_name, charm_href, talent_name, talent_level = charm

    use_charm = model.NewBoolVar(f"use_charm_{charm_name}")

    charm_talent_level = model.NewIntVar(
        lb=0,
        ub=4,
        name=f"charm_{charm_name}_talent_{talent_name}_level_{talent_level}",
    )

    model.Add(charm_talent_level == talent_level).only_enforce_if(use_charm)
    model.Add(charm_talent_level == 0).only_enforce_if(use_charm.Not())

    # Store variables
    use_equipement_vars["charm"][charm_name] = use_charm

    if talents_not_aggregated.get(talent_name) is None:
        talents_not_aggregated[talent_name] = []
    talents_not_aggregated[talent_name].append(charm_talent_level)

# Compute talent sums into proxy variables
for talent_name, talent_lvl_list in talents_not_aggregated.items():
    talent_sum = model.NewIntVar(0, 4, f"talent_sum_{talent_name}")

    # Map the sum
    model.Add(talent_sum == sum(talent_lvl_list))
    talents_aggregated[talent_name] = talent_sum

# Say that there is at most one charm
model.Add(sum(list(use_equipement_vars["charm"].values())) <= 1)


In [56]:
single_objective = talents_aggregated["Cercle de vie"]
# single_objective = talents_aggregated["Auto-amélioration"]

model.maximize(single_objective)
solver = cp_model.CpSolver()
status = solver.solve(model)

solver_statuses = {
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"Solver status: {solver_statuses[status]}")
# if status in (cp_model.OPTIMAL, status == cp_model.FEASIBLE):
#     print(solver.value(single_objective))
for charm_name, use_charm_variable in use_equipement_vars["charm"].items():
    if solver.value(use_charm_variable) == 1:
        print(charm_name)


Solver status: OPTIMAL
Talisman de récupération III


4

In [ ]:
model = cp_model.CpModel()

_vars = {
    "use_armor_piece_booleans": {},
    "talent_lists": {},
    "talent_sums": {},
    "jewel_emplacement_lists": {},
    "jewel_emplacement_sums": {},
    "jewel_emplacement_sums_total_armor": {},
}

unique_pieces = armor["piece"].unique().sort().to_list()

for armor_piece in unique_pieces:
    if _vars.get("use_armor_piece_booleans").get(armor_piece) is None:
        _vars["use_armor_piece_booleans"][armor_piece] = {}

    if _vars.get("jewel_emplacement_lists").get(armor_piece) is None:
        _vars["jewel_emplacement_lists"][armor_piece] = {}

    if _vars.get("jewel_emplacement_sums").get(armor_piece) is None:
        _vars["jewel_emplacement_sums"][armor_piece] = {}

    armor_piece_filtered = armor.filter(pl.col("piece") == armor_piece)
    unique_armor_pieces_names = armor_piece_filtered["name"].unique().sort().to_list()
    for unique_armor_piece_name in unique_armor_pieces_names:
        unique_armor_piece = armor_piece_filtered.filter(
            pl.col("name") == unique_armor_piece_name
        )

        # Define a boolean that tells if the armor piece is equipped
        names = unique_armor_piece["name"].to_list()
        name = names[0]
        armor_piece_equipped = model.NewBoolVar(f"use_piece_{armor_piece}_{name}")

        for row in unique_armor_piece.iter_rows():
            (
                _,
                name,
                talent_name,
                talent_level,
                jewel_0,
                jewel_1,
                jewel_2,
                jewel_3,
                jewel_4,
            ) = row
            # Create a variable that tells that the talent is active due to the fact that the armor piece is equipped
            var_talent_lvl = model.NewIntVar(
                lb=0,
                ub=30,
                name=f"talent_{talent_name}_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_talent_lvl == talent_level).only_enforce_if(
                armor_piece_equipped
            )
            model.Add(var_talent_lvl == 0).only_enforce_if(armor_piece_equipped.Not())

            # Create the key if it doesn't exist
            if _vars["talent_lists"].get(talent_name) is None:
                _vars["talent_lists"][talent_name] = []

            # Jewel emplacement part
            # LVL 1
            var_nb_jewel_1 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_1_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_1 == jewel_1).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_1 == 0).only_enforce_if(armor_piece_equipped.Not())

            # LVL 2
            var_nb_jewel_2 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_2_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_2 == jewel_2).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_2 == 0).only_enforce_if(armor_piece_equipped.Not())

            # LVL 3
            var_nb_jewel_3 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_3_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_3 == jewel_3).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_3 == 0).only_enforce_if(armor_piece_equipped.Not())

            # LVL 4
            var_nb_jewel_4 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_4_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_4 == jewel_4).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_4 == 0).only_enforce_if(armor_piece_equipped.Not())

            # Register variables
            _vars["talent_lists"][talent_name].append(var_talent_lvl)

        if _vars.get("jewel_emplacement_lists").get(armor_piece) is None:
            _vars["jewel_emplacement_lists"][armor_piece] = {}

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl1") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl1"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl1"].append(var_nb_jewel_1)

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl2") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl2"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl2"].append(var_nb_jewel_2)

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl3") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl3"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl3"].append(var_nb_jewel_3)

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl4") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl4"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl4"].append(var_nb_jewel_4)

        # Store the boolean that tells if the armor piece is equipped
        _vars["use_armor_piece_booleans"][armor_piece][name] = armor_piece_equipped

    # Create variables that are the sum of the jewel emplacements
    for i in range(1, 5):
        lvl = f"lvl{i}"
        var_jewel_emplacement_sums = model.NewIntVar(
            lb=0,
            ub=30,
            name=f"jewel_emplacement_sums_{lvl}_from_type_{armor_piece}",
        )
        model.Add(
            var_jewel_emplacement_sums
            == sum(_vars["jewel_emplacement_lists"][armor_piece][lvl])
        )
        _vars["jewel_emplacement_sums"][armor_piece][lvl] = var_jewel_emplacement_sums

    # Add the constraint of only one type of armor piece equipped at a time
    model.Add(sum(_vars["use_armor_piece_booleans"][armor_piece].values()) <= 1)

# Create a variable that symbolizes the total number of armor jewels
for i in range(1, 5):
    lvl = f"lvl{i}"
    var_jewel_emplacement_sums = model.NewIntVar(
        lb=0,
        ub=30,
        name=f"jewel_{lvl}_emplacement_sums_for_all_armor_pieces",
    )
    model.Add(
        var_jewel_emplacement_sums
        == sum(
            _vars["jewel_emplacement_sums"][armor_piece][lvl]
            for armor_piece in unique_pieces
        )
    )
    _vars["jewel_emplacement_sums_total_armor"][lvl] = var_jewel_emplacement_sums

# Create a variable that symbolizes the total number of weapon jewels
# ...

# Jewel optimization part
# ...

# Charm talents part
# ...

# Compute the talent sums
for talent_name, talent_vars in _vars["talent_lists"].items():
    var_talent_sum = model.NewIntVar(
        lb=0,
        ub=30,
        name=f"talent_sum_for_{talent_name}",
    )
    model.Add(var_talent_sum == sum(talent_vars))

    _vars["talent_sums"][talent_name] = var_talent_sum

# Add talent sum cap
# ...

# Add set bonus minmums
# ...


## Objective

# Add penalty for using too many jewels

# Add additional objective value for eventual addional talents (still minimize nb of jewels)

# Add talent objective weighting

# Add optional strict talent optimization
single_objective = _vars["talent_sums"]["Vengeance"]

model.maximize(single_objective)
solver = cp_model.CpSolver()
status = solver.solve(model)

solver_statuses = {
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"Solver status: {solver_statuses[status]}")
for armor_piece, var_dict in _vars["use_armor_piece_booleans"].items():
    for name, var in var_dict.items():
        if solver.value(var) == 1:
            print(f"{name} ({armor_piece})")
            display(armor.filter(pl.col("name") == name))


Solver status: OPTIMAL
Avant-bras Gypceros α (Bras)


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Bras""","""Avant-bras Gypceros α""","""Cuir souple""",1,3,0,0,0,0
"""Bras""","""Avant-bras Gypceros α""","""Vengeance""",2,3,0,0,0,0
"""Bras""","""Avant-bras Gypceros α""","""Toxicologie""",1,3,0,0,0,0


Talons putrides α (Jambes)


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Jambes""","""Talons putrides α""","""Sagesse transmise""",1,1,1,1,0,0
"""Jambes""","""Talons putrides α""","""Vengeance""",1,1,1,1,0,0
"""Jambes""","""Talons putrides α""","""Crâne d'acier""",2,1,1,1,0,0


Tassette Nu Udra β (Taille)


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Taille""","""Tassette Nu Udra β""","""Mutinerie du Nu Udra""",1,2,0,0,1,0
"""Taille""","""Tassette Nu Udra β""","""Rage supérieure""",1,2,0,0,1,0
"""Taille""","""Tassette Nu Udra β""","""Vengeance""",2,2,0,0,1,0


Cotte Ajarakan β (Torse)


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Torse""","""Cotte Ajarakan β""","""Pelage attrayant""",1,2,0,1,0,0
"""Torse""","""Cotte Ajarakan β""","""Vengeance""",2,2,0,1,0,0


Heaume Nu Udra α (Tête)


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Tête""","""Heaume Nu Udra α""","""Mutinerie du Nu Udra""",1,1,2,0,0,0
"""Tête""","""Heaume Nu Udra α""","""Faveur du seigneur""",1,1,2,0,0,0
"""Tête""","""Heaume Nu Udra α""","""Vengeance""",2,1,2,0,0,0
"""Tête""","""Heaume Nu Udra α""","""Contre-attaque""",1,1,2,0,0,0
